In [13]:
import logging
import os
from pathlib import Path
from typing import Any

from tqdm import tqdm
import pandas as pd
import numpy as np
import hydra
import lightning as L
import lightning.pytorch as pl
import peft
import torch
from datasets import DatasetDict, load_dataset
from hydra.utils import instantiate
from omegaconf import DictConfig, ListConfig, OmegaConf
from peft.tuners.lora import LoraLayer
from torch import Tensor, nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, default_data_collator

import peta
from peta.models.LinearizedModel import LinearizedModelWraper
from peta.utils import TimeIt, TitledLog
from peta.utils.logging.rich import pprint_yaml, setup_colorlogging
from peta.utils.ml.devices import num_devices

log = logging.getLogger(__name__)

# disable tokenizers parallelism
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from finetune_lm import load_model_from_config

In [8]:
def load_finetuned_model(
    model: str,
    dataset: str,
    finetune_mode: str,
    version: int,
):
    log_dir: Path = (
        Path("logs") / model / dataset / finetune_mode / f"version_{version}"
    )
    config_path = log_dir / "config.yaml"
    cfg: DictConfig = OmegaConf.load(config_path)

    # load model from config
    with TitledLog("load pretrained model and tokenizer", log_fn=log.info):
        _return = load_model_from_config(cfg)
        tokenizer: AutoTokenizer = _return["tokenizer"]
        model: AutoModelForSeq2SeqLM | peft.PeftModel = _return["model"]

        # load checkpoint
        checkpoint_dir = log_dir / "checkpoints"
        checkpoints = os.listdir(checkpoint_dir)
        # get checkpoint files with `step=2000.pth` in its name
        checkpoints = list(filter(lambda x: "step=2000.pth" in x, checkpoints))
        assert (
            len(checkpoints) == 1
        ), f"multiple checkpoints found, found checkpoints: {checkpoints}"
        checkpoint = checkpoints[0]
        log.info(f"load checkpoint from {checkpoint}")

        # load trainable parameters
        state_dict = torch.load(checkpoint_dir / checkpoint, map_location="cpu")
        model.load_state_dict(state_dict, strict=False)
        model.eval()

    return {
        "config": cfg,
        "model": model,
        "tokenizer": tokenizer,
    }

In [16]:
model = "flan-t5-base"
dataset = "glue-cola"
finetune_mode = "standard"

model = load_finetuned_model(model, dataset, finetune_mode, 0)
cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

datasets = instantiate(cfg.dataset.datasets)


if "validation" in datasets:
    val_dataset = datasets["validation"]
elif "validataion_matched" in datasets:
    # mnli
    val_dataset = datasets["validataion_matched"]
else:
    raise KeyError(datasets.keys())

# convert the task to text-to-text format
if hasattr(cfg.dataset, "preprocessor"):
    preprocessor = instantiate(
        cfg.dataset.preprocessor,
        tokenizer=tokenizer,
        tokenizer_kwargs=cfg.model.tokenizer_kwargs
        if hasattr(cfg.model, "tokenizer_kwargs")
        else None,
    )
    val_dataset = val_dataset.map(
        preprocessor,
        **cfg.dataset.map_kwargs if hasattr(cfg.dataset, "map_kwargs") else {},
    )

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    num_workers=0,
    shuffle=False,
    collate_fn=default_data_collator,
)

[21:40:08] INFO     =======load pretrained model and tokenizer========                              ]8;id=791281;file:///tmp/ipykernel_42254/501775352.py\501775352.py]8;;\:]8;id=272154;file:///tmp/ipykernel_42254/501775352.py#14\14]8;;\

[21:40:12] INFO     no peft config found, use full finetuning.                                   ]8;id=46763;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=258350;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#128\128]8;;\

           INFO     load checkpoint from epoch=3_step=2000.pth                                      ]8;id=763378;file:///tmp/ipykernel_42254/501775352.py\501775352.py]8;;\:]8;id=441232;file:///tmp/ipykernel_42254/501775352.py#28\28]8;;\

           INFO     ==================================================                              ]8;id=809651;file:///tmp/ipykernel_42254/501775352.py\501775352.py]8;;\:]8;id=891435;file:///tmp/ipykernel_42254/501775352.py#14\14]8;;\

Running tokenizer on dataset:   0%|          | 0/1043 [00:00<?, ? examples/s]

In [17]:
for batch_idx, batch in enumerate(val_loader):
    print(batch)
    break

ValueError: too many values to unpack (expected 2)